<a href="https://colab.research.google.com/github/engcnichida/codigoarduino/blob/main/Aumento_do_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import zipfile
import random
import numpy as np
import tensorflow as tf
import pandas as pd
import PIL
import shutil
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Baixando o Dataset
dataset_url = "https://github.com/RaptorMaps/InfraredSolarModules/raw/master/2020-02-14_InfraredSolarModules.zip"
zip_dir = tf.keras.utils.get_file(
    fname='InfraredSolarModules.zip', origin=dataset_url,
    cache_subdir='/content/drive/MyDrive/IA_PhotovoltaicPanel',
    extract=True, archive_format='zip', cache_dir='/content/drive/MyDrive')

# Lendo as classes do Dataset
classes_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/module_metadata.json"
classes = pd.read_json(classes_dir, orient='index')
class_names = classes.anomaly_class.unique()

# Criação das pastas por classes
classified_images_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages"
os.makedirs(classified_images_dir, exist_ok=True)

for class_name in class_names:
    class_dir = os.path.join(classified_images_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Separação das imagens por classes
images_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/images/"
target = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages/"

qtde = 0
for class_name in class_names:
    df_class = classes.loc[classes['anomaly_class'] == class_name]
    file_paths = df_class.iloc[:, 0]

    for file_path in file_paths:
        file_name = file_path.replace('images/', '')
        from_dir = os.path.join(images_dir, file_name)
        to_dir = os.path.join(target, class_name, file_name)
        shutil.copyfile(from_dir, to_dir)

    image_count = len(os.listdir(os.path.join(target, class_name)))
    print("Classe %s copiada com sucesso. Quantidade: %d" % (class_name, image_count))
    qtde += image_count

print("Imagens copiadas: %d" % qtde)

# Verificação da quantidade de imagens
data_dir = '/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages/'
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))

print(data_dir)
print("Quantidade: %d" % image_count)


# Função para calcular a porcentagem de imagens de cada classe
def calculate_class_percentage(data_dir):
    class_counts = {}
    total_count = 0

    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            file_count = len(os.listdir(class_dir))
            class_counts[class_name] = file_count
            total_count += file_count

    class_percentages = {}
    for class_name, count in class_counts.items():
        class_percentages[class_name] = count / total_count

    return class_percentages

classified_images_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages"
class_names = os.listdir(classified_images_dir)

for class_name in class_names:
    class_dir = os.path.join(classified_images_dir, class_name)
    image_count = len(os.listdir(class_dir))


# Função para gerar imagens aumentadas para classes com baixa quantidade de imagens
def generate_augmented_images(data_dir, class_percentages, target_percentage, img_height, img_width):
    # Determina a quantidade de imagens da classe de referência
    reference_class = max(class_percentages, key=class_percentages.get)
    reference_count = len(os.listdir(os.path.join(data_dir, reference_class)))
    target_count = int(target_percentage * reference_count)

    # Cria o gerador de imagens para aumento de dados
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for class_name, percentage in class_percentages.items():
        class_dir = os.path.join(data_dir, class_name)

        # Verifica se a classe possui uma quantidade baixa de imagens
        if percentage < target_percentage:
            class_images = os.listdir(class_dir)
            num_images_to_generate = target_count - len(class_images)

            # Gera imagens aumentadas para igualar a quantidade da classe de referência
            for i in range(num_images_to_generate):
                image_file = random.choice(class_images)
                image_path = os.path.join(class_dir, image_file)

                # Carrega a imagem
                image = tf.io.read_file(image_path)
                image = tf.image.decode_jpeg(image, channels=3)

                # Redimensiona a imagem
                image = tf.image.resize(image, [img_height, img_width])

                # Aplica transformações de aumento de dados
                augmented_images = []
                for j in range(3):  # Gera 3 imagens aumentadas para cada imagem original
                    augmented_image = datagen.random_transform(np.array(image))
                    augmented_images.append(augmented_image)

                # Salva as imagens aumentadas
                for j, augmented_image in enumerate(augmented_images):
                    augmented_image_path = os.path.join(class_dir, f'augmented_{i}_{j}.jpg')
                    augmented_image = tf.cast(augmented_image, tf.uint8)
                    augmented_image = tf.image.encode_jpeg(augmented_image)
                    tf.io.write_file(augmented_image_path, augmented_image)



# Diretórios
data_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages"
output_dir = "/content/drive/MyDrive/IA_PhotovoltaicPanel/AugmentedDataset"

# Configurações
img_height = 224
img_width = 224
target_percentage = 0.3

# Calcula a porcentagem de imagens em cada classe
class_percentages = calculate_class_percentage(data_dir)

# Verifica se é necessário gerar imagens aumentadas
if min(class_percentages.values()) < target_percentage:
    # Gera imagens aumentadas para as classes com baixa quantidade de imagens
    generate_augmented_images(data_dir, class_percentages, target_percentage, img_height, img_width)
    print("Imagens aumentadas geradas com sucesso!")

# Copia todas as imagens para um diretório de saída
shutil.copytree(data_dir, output_dir)
print("Dataset final copiado para o diretório de saída:", output_dir)

Classe No-Anomaly copiada com sucesso. Quantidade: 10000
Classe Cell copiada com sucesso. Quantidade: 1877
Classe Hot-Spot copiada com sucesso. Quantidade: 249
Classe Offline-Module copiada com sucesso. Quantidade: 827
Classe Vegetation copiada com sucesso. Quantidade: 1639
Classe Diode copiada com sucesso. Quantidade: 1499
Classe Shadowing copiada com sucesso. Quantidade: 1056
Classe Cracking copiada com sucesso. Quantidade: 940
Classe Diode-Multi copiada com sucesso. Quantidade: 175
Classe Hot-Spot-Multi copiada com sucesso. Quantidade: 246
Classe Cell-Multi copiada com sucesso. Quantidade: 1288
Classe Soiling copiada com sucesso. Quantidade: 204
Imagens copiadas: 20000
/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/ClassifiedImages
Quantidade: 20000
Imagens aumentadas geradas com sucesso!
Dataset final copiado para o diretório de saída: /content/drive/MyDrive/IA_PhotovoltaicPanel/AugmentedDataset


# Função para gerar imagens aumentadas para classes com baixa quantidade de imagens
def generate_augmented_images(data_dir, output_dir, class_name, class_images, target_count, img_height, img_width):
    os.makedirs(output_dir, exist_ok=True)

    # Cria o gerador de imagens para aumento de dados
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    output_class_dir = os.path.join(output_dir, class_name)
    os.makedirs(output_class_dir, exist_ok=True)

    # Gera imagens aumentadas para igualar a quantidade da classe de referência
    for i in range(target_count - len(class_images)):
        image_file = random.choice(class_images)
        image_path = os.path.join(data_dir, class_name, image_file)

        # Carrega a imagem
        image = PIL.Image.open(image_path)
        image = image.convert('RGB')
        image = np.array(image)

        # Redimensiona a imagem
        image = tf.image.resize(image, [img_height, img_width])

        # Aplica transformações de aumento de dados
        augmented_images = []
        for j in range(3):  # Gera 3 imagens aumentadas para cada imagem original
            augmented_image = datagen.random_transform(image)
            augmented_images.append(augmented_image)

        # Salva as imagens aumentadas
        for j, augmented_image in enumerate(augmented_images):
            augmented_image_path = os.path.join(output_class_dir, f'{i}_{j}.jpg')
            augmented_image = PIL.Image.fromarray(augmented_image)
            augmented_image.save(augmented_image_path)


# Diretórios de entrada e saída
data_dir = '/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules'
output_dir = '/content/drive/MyDrive/IA_PhotovoltaicPanel/AugmentedDataset'

# Parâmetros
img_height = 40
img_width = 24
target_percentage = 0.8

# Lê as classes do dataset
classes_dir = '/content/drive/MyDrive/IA_PhotovoltaicPanel/InfraredSolarModules/module_metadata.json'
classes = pd.read_json(classes_dir, orient='index')
class_names = classes.anomaly_class.unique()

# Calcula as porcentagens de cada classe
class_percentages = calculate_class_percentage(data_dir)
print(class_percentages)

# Verifica as classes que precisam de aumento de dados
classes_requiring_augmentation = [
    class_name for class_name, percentage in class_percentages.items()
    if percentage < target_percentage
]

if len(classes_requiring_augmentation) > 0:
    # Gera imagens aumentadas para as classes que precisam
    for class_name in classes_requiring_augmentation:
        class_dir = os.path.join(data_dir, class_name)
        class_images = os.listdir(class_dir)
        target_count = int(target_percentage * len(class_images))

        generate_augmented_images(data_dir, output_dir, class_name, class_images, target_count, img_height, img_width)

    # Imprime a quantidade de imagens em cada classe após o aumento
    for class_name in os.listdir(output_dir):
        class_dir = os.path.join(output_dir, class_name)
        if os.path.isdir(class_dir):
            num_images = len(os.listdir(class_dir))
            print(f"Classe: {class_name} - Quantidade de imagens: {num_images}")

    # Compacta o diretório de saída em um arquivo zip
    zip_path = '/content/drive/MyDrive/IA_PhotovoltaicPanel/augmented_dataset.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, arcname)

    print('Dataset aumentado e salvo em:', zip_path)
else:
    print('Não é necessário aumentar o dataset, todas as classes atendem ao critério de porcentagem.')
